you should first git concerto with
git clone git@github.com:melobio/Concerto-reproducibility.git
add an path to it

In [3]:
import os
import sys
import numpy as np
import scanpy as sc
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


sys.path.append("Concerto-reproducibility")
from concerto_function5_3 import *
from os.path import join as pj

os.environ['CUDA_VISIBLE_DEVICES']='1'

In [4]:
#Select an available GPU to run on a multi-GPU computer or you can run it directly on the CPU without executing this cell
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-05-12 12:28:44.056114: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-12 12:28:44.079899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:40:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 1.03TiB/s
2023-05-12 12:28:44.079974: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-12 12:28:44.085879: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-12 12:28:44.086104: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-05-12 12:28:44.088860: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [5]:
task = 'wnn'
experiment = 'concerto_offline'
result_path = pj('../../../result', task, experiment)
data_root = '/opt/data/private/xx/code/MIRACLE/data/raw/rna+adt'
save_path = '../concerto/offline_cellmask/'

### read data (h5ad)

In [6]:
adata_RNA = sc.read_h5ad(pj(data_root, task,'rna_cellmask.h5ad'))
adata_Protein = sc.read_h5ad(pj(data_root, task,'adt_cellmask.h5ad'))

### process

In [7]:
adata_RNA = preprocessing_rna(adata_RNA,min_features = 0, is_hvg=False,batch_key='batch')
adata_Protein = preprocessing_rna(adata_Protein,min_features = 0, is_hvg=False,batch_key='batch')

Processed dataset shape: (46213, 3617)
Processed dataset shape: (46213, 224)


In [8]:
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
adata_RNA = sc.AnnData(X=adata_RNA.X, obs=adata_RNA.obs[['l1', 'batch']])
adata_Protein = sc.AnnData(X=adata_Protein.X, obs=adata_Protein.obs[['l1', 'batch']])

adata_RNA.write_h5ad(save_path + 'adata_RNA.h5ad')
adata_Protein.write_h5ad(save_path + 'adata_Protein.h5ad')

### make TF-record
TF-record is a binary file containing sequences of serialized byte-strings for the sharding file in TensorFlow, which is especially suitable for streaming data input network.

In [9]:
RNA_tf_path = save_path + 'tfrecord/RNA_tf/'
Protein_tf_path = save_path + 'tfrecord/Protein_tf/'

In [13]:
RNA_tf_path = concerto_make_tfrecord(adata_RNA,tf_path = save_path + 'tfrecord/RNA_tf/',batch_col_name = 'batch')
Protein_tf_path = concerto_make_tfrecord(adata_Protein,tf_path = save_path + 'tfrecord/Protein_tf/',batch_col_name = 'batch')

AnnData object with n_obs × n_vars = 46213 × 3617
    obs: 'l1', 'batch'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
counter: 7000 shape: (3617,), batch: 1
counter: 8000 shape: (3617,), batch: 1
counter: 9000 shape: (3617,), batch: 1
counter: 10000 shape: (3617,), batch: 1
counter: 11000 shape: (3617,), batch: 2
counter: 12000 shape: (3617,), batch: 2
counter: 13000 shape: (3617,), batch: 2
counter: 14000 shape: (3617,), batch: 2
counter: 15000 shape: (3617,), batch: 3
counter: 16000 shape: (3617,), batch: 3
counter: 17000 shape: (3617,), batch: 3
counter: 18000 shape: (3617,), batch: 3
counter: 19000 shape: (3617,), batch: 3
counter: 20000 shape: (3617,), batch: 4
counter: 21000 shape: (3617,), batch: 4
counter: 22000 shape: (3617,), batch: 4
counter: 23000 shape: (3617,), batch: 4
counter: 24000 s

### train (multimodal integration)

In [14]:
weight_path = save_path + 'weight/'
RNA_tf_path = save_path + 'tfrecord/RNA_tf/'
Protein_tf_path = save_path + 'tfrecord/Protein_tf/'
concerto_train_multimodal(RNA_tf_path,Protein_tf_path,weight_path,super_parameters={'batch_size': 64, 'epoch_pretrain': 5, 'lr': 1e-4,'drop_rate': 0.1})

../concerto/offline_cellmask/tfrecord/RNA_tf/tf_4.tfrecord
../concerto/offline_cellmask/tfrecord/Protein_tf/tf_4.tfrecord
Epoch 1, step 5, simclr loss: 11.1957.
Epoch 1, step 10, simclr loss: 11.3559.
Epoch 1, step 15, simclr loss: 11.3526.
Epoch 1, step 20, simclr loss: 11.3083.
Epoch 1, step 25, simclr loss: 11.1806.
Epoch 1, step 30, simclr loss: 11.0542.
Epoch 1, step 35, simclr loss: 10.9583.
Epoch 1, step 40, simclr loss: 10.8608.
Epoch 1, step 45, simclr loss: 10.7491.
Epoch 1, step 50, simclr loss: 10.6704.
Epoch 1, step 55, simclr loss: 10.5945.
Epoch 1, step 60, simclr loss: 10.5165.
Epoch 1, step 65, simclr loss: 10.4226.
../concerto/offline_cellmask/tfrecord/RNA_tf/tf_0.tfrecord
../concerto/offline_cellmask/tfrecord/Protein_tf/tf_0.tfrecord
Epoch 1, step 5, simclr loss: 9.8087.
Epoch 1, step 10, simclr loss: 9.6948.
Epoch 1, step 15, simclr loss: 9.4806.
Epoch 1, step 20, simclr loss: 9.3300.
Epoch 1, step 25, simclr loss: 9.2283.
Epoch 1, step 30, simclr loss: 9.1380.
Epoc

### test

In [24]:
weight_path = save_path + 'weight/'
RNA_tf_path = save_path + 'tfrecord/RNA_tf/'
Protein_tf_path = save_path + 'tfrecord/Protein_tf/'
saved_weight_path = save_path + './weight/weight_encoder_epoch5.h5'# You can choose a trained weight or use None to default to the weight of the last epoch.
embedding,batch,RNA_id,attention_weight =  concerto_test_multimodal(weight_path,RNA_tf_path,Protein_tf_path,n_cells_for_sample = None,super_parameters={'batch_size': 128, 'epoch_pretrain': 1, 'lr': 1e-4,'drop_rate': 0.1},saved_weight_path = saved_weight_path)

load saved weight
../concerto/offline_cellmask/tfrecord/RNA_tf/tf_4.tfrecord
../concerto/offline_cellmask/tfrecord/Protein_tf/tf_4.tfrecord
all_samples num:128
all_samples num:256
all_samples num:384
all_samples num:512
all_samples num:640
all_samples num:768
all_samples num:896
all_samples num:1024
all_samples num:1152
all_samples num:1280
all_samples num:1408
all_samples num:1536
all_samples num:1664
all_samples num:1792
all_samples num:1920
all_samples num:2048
all_samples num:2176
all_samples num:2304
all_samples num:2432
all_samples num:2560
all_samples num:2688
all_samples num:2816
all_samples num:2944
all_samples num:3072
all_samples num:3200
all_samples num:3328
all_samples num:3456
all_samples num:3584
all_samples num:3712
all_samples num:3840
all_samples num:3968
all_samples num:4096
all_samples num:4205
../concerto/offline_cellmask/tfrecord/RNA_tf/tf_0.tfrecord
../concerto/offline_cellmask/tfrecord/Protein_tf/tf_0.tfrecord
all_samples num:128
all_samples num:256
all_samples 

In [25]:
new_adata = adata_RNA[RNA_id]
new_adata.obsm['emb'] = embedding
new_adata
new_adata.write(save_path + './concerto_offline_cellmask.h5ad')

### save embedding

In [26]:
for i in pd.unique(new_adata.obs['batch']):
    if not os.path.exists(pj(result_path, 'default/predict','subset_%d/z/joint'%(i))):
        os.makedirs(pj(result_path, 'default/predict','subset_%d/z/joint'%(i)))
    pd.DataFrame(new_adata[new_adata.obs['batch']==i].obsm['emb']).to_csv(pj(result_path, 'default/predict','subset_%d/z/joint'%(i),'00.csv'), index=False, header=False)